In [ ]:
# Install aba Kasbi all those modules (python 2.7)
gensim==3.8.3
matplotlib==3.1.3
hydralit-components==1.0.9
matplotlib==3.1.3
pandas==1.3.5
plotly==4.14.3
pyLDAvis==3.3.1
pyspellchecker==0.6.3
regex==2022.3.15
scikit-learn==0.22.1
seaborn==0.11.2
smmap==5.0.0
spacy==3.2.4
streamlit==1.2.0
vaderSentiment==3.3.2
wordcloud==1.8.1

In [ ]:
df = pd.read_csv(r"C:/Users/sc84998/Downloads/gg_full_clean_dataset.zip")
df.shape

##### Removing empty values 

In [ ]:
df.isnull().sum()

In [ ]:
df.dropna(subset=['XXX'], inplace=True, axis=0)
df.reset_index(drop=True, inplace=True)

In [ ]:
df.isnull().sum()

## Tokenizing 🕵🏻‍♂

In [ ]:
# insttalih mn had lien : https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-3.3.0/en_core_web_lg-3.3.0.tar.gz
nlp = spacy.load('C:/nltk_data/!/en_core_web_lg/en_core_web_lg-3.2.0')

In [ ]:
# Tokenizer
tokenizer = Tokenizer(nlp.vocab)

# Tokenizer function
def tokenize(text):
    """
    Parses a string into a list of semantic units (words)
    Args:
        text (str): The string that the function will tokenize.
    Returns:
        list: tokens parsed out
    """
    text = str(text)
    # Removing url's
    pattern = r"http\S+"
    
    tokens = re.sub(pattern, "", text) # https://www.youtube.com/watch?v=O2onA4r5UaY
    tokens = re.sub('[^a-zA-Z 0-9]', '', text)
    tokens = re.sub('[%s]' % re.escape(string.punctuation), '', text) # Remove punctuation
    tokens = re.sub('\w*\d\w*', '', text) # Remove words containing numbers
    tokens = re.sub('@*!*\$*', '', text) # Remove @ ! $
    tokens = tokens.strip(',') # TESTING THIS LINE
    tokens = tokens.strip('?') # TESTING THIS LINE
    tokens = tokens.strip('!') # TESTING THIS LINE
    tokens = tokens.strip("'") # TESTING THIS LINE
    tokens = tokens.strip(".") # TESTING THIS LINE

    #tokens = tokens.lower().split() # Make text lowercase and split it
    
    return tokens

# Apply tokenizer
df['clean_en_corrct_acro_free_comments'] = df['XXX'].apply(tokenize)

# View those tokens (the 4th column)
df.tail()

In [ ]:
"""
Import Gensim and Wordcloud to use their stopwords as well and use the combined stopwords of ALL as the variable:
ALL_STOP_WORDS
"""

# Tokenizer
tokenizer = Tokenizer(nlp.vocab)


# Custom stopwords
custom_stopwords = ['hi','\n','\n\n', '&amp;', ' ', '.', '-', 'got', "it's", 'it’s', "i'm", 'i’m', 'im', 'want', 'like',
                    '$', '@','vehicle','camber','beautiful','friends','friend','mesh']

# Customize stop words by adding to the default list
STOP_WORDS = nlp.Defaults.stop_words.union(custom_stopwords)

# ALL_STOP_WORDS = spacy + gensim + wordcloud
ALL_STOP_WORDS = STOP_WORDS.union(SW).union(stopwords)


tokens = []

for doc in tokenizer.pipe(df['clean_en_corrct_acro_free_comments'], batch_size=500):
    doc_tokens = []    
    for token in doc:
        if token.text.lower() not in STOP_WORDS:
            doc_tokens.append(token.text.lower()) 
    tokens.append(doc_tokens)
type(tokens)
# Makes tokens column
df['tokens'] = tokens

# View df
df['tokens']

## Lemmatization🇬🇧

In [ ]:
# Make tokens a string again
df['tokens_back_to_text'] = [' '.join(map(str, l)) for l in df['tokens']]

def get_lemmas(text):
    '''Used to lemmatize the processed tweets'''
    lemmas = []
    doc = nlp(text)
    
    # Something goes here :P
    for token in doc: 
        if ((token.is_stop == False) and (token.is_punct == False)) and (token.pos_ != 'PRON'):
            lemmas.append(token.lemma_)
    
    return lemmas

df['lemmas'] = df['tokens_back_to_text'].apply(get_lemmas)

# Make lemmas a string again
df['lemmas_back_to_text'] = [' '.join(map(str, l)) for l in df['lemmas']]
# df[['original_tweet', 'lemmas_back_to_text']]

df

## Topic Modeling ㊙️ LDA

In [ ]:
# Create a id2word dictionary
id2word = Dictionary(df['lemmas'])
print(len(id2word))

In [ ]:
# Creating a corpus object 
corpus = [id2word.doc2bow(d) for d in df['lemmas']]
print(len(corpus))

In [ ]:
# Instantiating a LDA model 
base_model = LdaMulticore(corpus=corpus, num_topics=12, id2word=id2word, workers=12, passes=5,eta='auto')

In [ ]:
# Filtering for words 
words = [re.findall(r'"([^"]*)"',t[1]) for t in base_model.print_topics()]
# Create Topics
topics = [' '.join(t[0:10]) for t in words]

In [ ]:
# Getting the topics
for id, t in enumerate(topics):
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

In [ ]:
# Compute Perplexity
# a measure of how good the model is. lower the better
base_perplexity = base_model.log_perplexity(corpus)
print('\nPerplexity: ', base_perplexity) 

# Compute Coherence Score
coherence_model = CoherenceModel(model=base_model, texts=df['lemmas'], 
                                   dictionary=id2word, coherence='c_v')
coherence_lda_model_base = coherence_model.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_base)

## Topic Modeling ㊙️ Mallet LDA

In [ ]:
import os

In [ ]:
# insttalih tahowa via: https://mallet.cs.umass.edu/download.php
os.environ.update({'MALLET_HOME':r'C:/Users/sc84998/mallet-2.0.8/'}) 
path_to_mallet = 'C:/Users/sc84998/mallet-2.0.8/bin/mallet'  # CHANGE THIS TO YOUR MALLET PATH

In [ ]:
ldamallet_1_4 = gensim.models.wrappers.LdaMallet(path_to_mallet, 
                                             corpus=corpus, 
                                             num_topics=13,
                                             id2word=id2word,
                                             random_seed=42,
                                             alpha=10,
                                             workers=6)

In [ ]:
# Filtering for words 
words_mallet_1_4 = [re.findall(r'"([^"]*)"',t[1]) for t in ldamallet_1_4.print_topics()]

# Create Topics
topics_mallet_1_4 = [' '.join(t[0:10]) for t in words_mallet_1_4]

# Getting the topics
for id, t in enumerate(topics_mallet_1_4): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

In [ ]:
# Compute Coherence Score
coherence_model_mallet_1_4 = CoherenceModel(model=ldamallet_1_4, texts=df['lemmas'], 
                                   dictionary=id2word, coherence='c_v')
coherence_lda_model_mallet_1_4 = coherence_model_mallet_1_4.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_mallet_1_4)

### Topics Assignment

In [ ]:
topics_docs = []

for m in ldamallet_1_4[corpus]:
    topics_docs.append(m)

In [ ]:
topics = []
for td in topics_docs:
    max_prob = 0
    for p in td:
        if p[1] > max_prob:
            max_prob = p[1]
            max_prob_index = p[0]
    topics.append(max_prob_index)
df['topic'] = topics

### topics interpretation

In [ ]:
TOPICS_LABELS = ['AMI MODERNIZATION','LABEL MANUFACTURER','START & CHARGE','WHEEL NOISE GLASS','WATER ENTRANCE','BLUETOOTH BOX',
                 'DOOR OPENING & CLOSING','DOOR STUCK BUTTON','START & CHARGE','rear tire hyd','COMPAGN BRAKE','LIGHT INDICATOR',
                 'charge cord long','charge customer lock']
df['TOPIC_LABEL'] = [TOPICS_LABELS[int(topic)] for topic in df['topic']]

In [ ]:
result = df[["en_corrct_acro_free_comments","TOPIC_LABEL"]]
result.head(20)

In [ ]:
result['TOPIC_LABEL'].value_counts().to_frame()

### Model querying

In [1]:
query = 'door'
query_vec_bow = id2word.doc2bow(query.lower().split())

# convert the query to Mallet space
vec_mallet = ldamallet_1_4[query_vec_bow]  
# This shows how the query relates to the 10 topics
vec_mallet

NameError: name 'id2word' is not defined

In [ ]:
from gensim import similarities
index = similarities.MatrixSimilarity(ldamallet_1_4[corpus]) # transform corpus to Mallet space and index it

sims = index[vec_mallet]  # perform a similarity query against the corpus

sims = sorted(enumerate(sims), key=lambda item: -item[1])

In [ ]:
similarity_result = []
for i, s in enumerate(sims):
    if s[1] >= 0.97:
        #print('doc number:',s[0],'   has similarity of',s[1])
        similarity_result.append(tuple(df.loc[[s[0]],['tokens_back_to_text','TOPIC_LABEL']].values[0]))

In [ ]:
print("Similarity of query:",query,"\n")
df_similarity = pd.DataFrame(similarity_result, 
                 columns =['CLIENT_COMMENT', 'TOPIC_DETECTED'])
df_similarity[100:120]